# Preparazione dell'ambiente di sviluppo 
- importiamo le librerie necessarie all'analisi

- se necessario vi manderemo un link per rieseguire da casa il notebook su cui lavorerete oggi, via browser, senza dover installare nulla (ci stiamo lavorando)
https://mybinder.org/v2/gh/davidpelosi21/ICD2025.git/HEAD

In [1]:
# lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from IPython.display import Image
from sunpy.net import Fido
from sunpy.net import attrs as a

## Le basi di Neutron Monitor (NM)
- questa mappa mostra il network globale di basi di Neutron Monitor disponibili 

In [ ]:
Image(filename='figures/stations.png', width=800, height=800)

Se non trovate qualche base di NM nella tabella sotto (può capitare) - cercate le informazioni qui:
[Link to Neutron Monitor Database](https://www.nmdb.eu/station/)

# Questa tabella mostra le proprietà principali di ciascun NM, in partcolare:
- coordinate geografiche
- altitudine (un parametro importante per identificare un GLE)
- il cutoff geomagnetico (espresso in GigaVolt) 

In [ ]:
csv_file_path = 'neutron_monitor_stations.csv'
df = pd.read_csv(csv_file_path)
df

## Esercizio: Osservazione di un GLE 

Abbiamo visto come i neutron monitor siano in grado di osservare (indirettamente) un evento solare intenso (flare). L'effetto si manifesta come un rapido aumento dei conteggi.

Questo effetto viene chiamato **Ground Level Enhancement 'GLE'** 

# Ognuno di voi potrà scegliere un evento tra i seguenti:
- event 70
- event 71
- event 72
- event 73
- event 74 (esempio)
- event 75
- event 76

## Verificare se nel periodo analizzato si è verificato un evento solare intenso, osservando i conteggi di diverse basi di Neutron Monitor (in realtà la loro variazione rispetto alla meda del periodo precendente)


- Un evento GLE viene registrato quando ci sono aumenti coincidenti nel tempo e statisticamente significativi dei tassi di conteggio di **almeno due monitori di neutroni situati in posizioni differenti**, tra cui almeno un monitor di neutroni vicino al **livello del mare** (Ground-level) e un corrispondente aumento nel **flusso di protoni misurato da un esperimento spaziale** (Poluianov et al., 2017).

# Sub-GLE
osservare un aumento significativo dei conteggi in una base di NM è necessaario ma non sufficiente per affermare di aver osservato un GLE. Esistono anche i sub-GLE, ossia fenomeni meno intensi che non raggiungono i NM posti a basse altitudini, inoltre improvvisi malfunzionamenti dell'hardware di un NM potrebbero dare un falso segnale

In [ ]:
Image(filename='figures/GLE_ex.png', width=800, height=800)
# hourly data

# Scegli il tuo evento

In [5]:
event = ''
# e.g. '74'

## Seleziona i dati di Neutron Monitor da visualizzare

In [6]:
base1 = '' 
# per aggiungere piu basi da visualizzare (uncomment)
#base2 = 'nain'
# base3 = '...'
# base4 = '...'

In [7]:
nm1 = f'event_{event}/c0{event}{base1}.dat'

# per aggiungere piu basi da visualizzare (uncomment)
nm2 = f'event_{event}/c0{event}{base2}.dat'
# nm3 = f'event_74/c074{base3}.dat'
# nm4 = f'event_74/c074{base4}.dat'

In [8]:
data_nm1 = pd.read_csv(nm1, skiprows=9, header=None,sep='\s+', on_bad_lines='skip')
ind1 = data_nm1[data_nm1.apply(lambda row: row.astype(str).str.contains('\*').any(), axis=1)].index
data_nm1 = data_nm1.iloc[:ind1[0]]
data_nm1 = data_nm1.replace(-999.00, np.nan)
data_nm1 = data_nm1.dropna()
data_nm1 = data_nm1.drop(data_nm1[data_nm1[8] == '-9999'].index)



data_nm1['DateTime'] = pd.to_datetime(data_nm1[1].astype(str) + ' ' + data_nm1[3].str[:6], format='%y%m%d %H%M%S')
data_nm1['Value'] = data_nm1[8].astype(float)


# per aggiungere piu basi da visualizzare (uncomment)

# data_nm2 = pd.read_csv(nm2, skiprows=9, header=None,sep='\s+', on_bad_lines='skip')
# ind2 = data_nm2[data_nm2.apply(lambda row: row.astype(str).str.contains('\*').any(), axis=1)].index
# data_nm2 = data_nm2.iloc[:ind2[0]]
# data_nm2 = data_nm2.replace(-999.00, np.nan)
# data_nm2 = data_nm2.dropna()
# data_nm2 = data_nm2.drop(data_nm2[data_nm2[8] == '-9999'].index)
# data_nm2['DateTime'] = pd.to_datetime(data_nm2[1].astype(str) + ' ' + data_nm2[3].str[:6], format='%y%m%d %H%M%S')
# data_nm2['Value'] = data_nm2[8].astype(float)



# Abbiamo caricato i dati, siamo pronti per visualizzare

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(data_nm1['DateTime'], data_nm1['Value'], label=f'{base1}')
#plt.plot(data_nm2['DateTime'], data_nm2['Value'], label=f'{base2}')
#plt.plot(data_nm3['DateTime'], data_nm3['Value'], label=f'{base3}')
#plt.plot(data_nm4['DateTime'], data_nm4['Value'], label=f'{base4}')
plt.xlabel('Time UTC')
plt.ylabel('Relative to pre-increase, %')
# title the year of the event
plt.title(f'Event {event} -- Year =  {data_nm1["DateTime"][0].year}')
plt.legend()
plt.grid(True)
plt.show()

# Se nel perido osservato si identifica un GLE (secondo la definizione)
- stima la data di arrivo sulla Terra dell'evento solare:
- verifica se nella stessa data un esperimento spaziale ha misurato un aumento del flusso di protoni (solar flare)

In [16]:
data_evento = ''
# e.g. 'year-month-day'

## Osservazione del flusso di protoni dallo spazio 

oltre alle osservazioni a terra è necessario verificare se un esperimento spaziale ha osservato (nello stesso periodo) un aumento del flusso di protoni

## Dati da Satellite
Per verificare il corrispondente aumento nel flusso di protoni misurato da un esperimento spaziale, usiamo i dati di **GOES** (Geostationary Operational Environmental Satellite)

In [ ]:
Image(filename='figures/goes.png', width=800, height=800)

# Cosa vede GOES?

In [ ]:
Image(filename='figures/GOES_proton.png', width=800, height=800)

# Osservazione nell'ottico
Quando GOES osserva un aumento del flusso di protoni, associato ad un solar flare, classifica l'evento in base all'intensità misurata nella banda ottica

## Classificazione dei solar flare in base alla loro intensità
| Solar Flare Class | Intensity $W/m^2$  |
|-------------------|------------------------------|
| B                 | $I < 10^{-6} $            |
| C                 | $ 10^{-6} < I < 10^{-5} $  |
| M                 | $ 10^{-5} < I < 10^{-4} $  |
| X                 | $ I > 10^{-4} $            |

In [ ]:
Image(filename='figures/GOES_photon.png', width=800, height=800)

## Verifica se nel giorno selezionato è stato osservato un flare solare da GOES

In [20]:
tstart = data_evento + ' 00:00:00'
tend = data_evento + ' 23:59:59'

In [ ]:
event_type = "FL"
result = Fido.search(a.Time(tstart, tend),
                     a.hek.EventType(event_type),
                     a.hek.FL.GOESCls > "M1.0",
                     a.hek.OBS.Observatory == "GOES")



# It"s also possible to access the HEK results from the
# `~sunpy.net.fido_factory.UnifiedResponse` by name.
hek_results = result["hek"]

filtered_results = hek_results["event_starttime", "event_peaktime",
                               "event_endtime", "fl_goescls", "ar_noaanum"]

by_magnitude = sorted(filtered_results, key=lambda x: ord(x['fl_goescls'][0]) + float(x['fl_goescls'][1:]), reverse=True)

for flare in by_magnitude:
    print(f"Class {flare['fl_goescls']} occurred on {flare['event_starttime']}")

# Domande
- nel periodo selezionato è avvento un GLE? 
- quali basi di NM avete usato per identificare il GLE?
- quale (o quali) solar flare (classe di intensità) è associato al GLE?